<a href="https://colab.research.google.com/github/umais2005/DeepLearning/blob/main/Keras_Tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
!pip install keras-tuner
import keras_tuner as kt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 1.4 MB/s eta 0:00:00


In [ ]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()


11490434/11490434 [==============================] - 0s 0us/step


In [ ]:
def build_model(hp:kt.HyperParameters):
  n_units = hp.Int("n_units", min_value=50, max_value=300, default=30)
  n_hidden = hp.Int("n_hidden", min_value=1, max_value=8, default=2)
  optimizer = hp.Choice(name='optimizer', values=['adam', 'sgd'])

  model = tf.keras.Sequential()
  model.add(tf.keras.layers.Input((28,28)))
  model.add(tf.keras.layers.Flatten())
  for _ in range(n_hidden):
    model.add(tf.keras.layers.Dense(n_units, activation='relu'))
  model.add(tf.keras.layers.Dense(10, activation='softmax'))

  model.compile(optimizer=optimizer,loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics=['accuracy'],)

  return model


In [ ]:
class mnist_hyper_model(kt.HyperModel):
  def build(self, hp):
    return build_model(hp)

  def fit(self, hp, model,*args,  **kwargs):
    early_stopper = tf.keras.callbacks.EarlyStopping(patience=5,monitor='val_accuracy', restore_best_weights=True)
    if 'callbacks' in kwargs:
        kwargs['callbacks'].append(early_stopper)
    else:
        kwargs['callbacks'] = [early_stopper]
    batch_size = hp.Int('batch_size', 16,128,sampling='log')
    return model.fit(*args, batch_size=batch_size, **kwargs)



In [ ]:
rnd_search = kt.RandomSearch(mnist_hyper_model(), 'val_accuracy',seed=42,overwrite=True,)

In [ ]:
rnd_search.search(X_train,y_train, epochs=50, validation_split=0.16)

Trial 10 Complete [00h 05m 35s]
val_accuracy: 0.9751041531562805

Best val_accuracy So Far: 0.9786458611488342
Total elapsed time: 00h 33m 03s


In [ ]:
best_model = rnd_search.get_best_models()[0]

In [ ]:
best_params = rnd_search.get_best_hyperparameters()[0]

In [ ]:
best_params.values

{'n_units': 118, 'n_hidden': 8, 'optimizer': 'adam', 'batch_size': 50}

In [ ]:
best_model.evaluate(X_test,y_test)

313/313 [==============================] - 1s 3ms/step - loss: 0.1524 - accuracy: 0.9754


[0.1524296998977661, 0.9753999710083008]